<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LayerizedNet-Tutorial" data-toc-modified-id="LayerizedNet-Tutorial-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><code>LayerizedNet</code> Tutorial</a></span><ul class="toc-item"><li><span><a href="#Create-layers-with-extra-logic" data-toc-modified-id="Create-layers-with-extra-logic-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Create layers with extra logic</a></span></li></ul></li></ul></div>

# `LayerizedNet` Tutorial

For many experiments, it is useful to have an explicit layerized version of your model.
You can do this by creating your network class directly like this, but then you can't use your new implemented functionality with pretrained models. `LayerizedNet` is a class that builds a copy of an existing framework into a network that is explicitly layerized, i.e. has a `layers` attribute, into a set of layers you define.

The functionality of layers can be plugged using multiple inheritance. This makes usage of `super()`s method resolution order. If you not familiar with it, it may be helpful to google it. The most important thing to note is, that `super()` does not always call the parent class of the class where it is used in the code: if an object of class `C(A, B)` calls `super()`, it looks for the method in class `A`. If this method uses `super()` itself, it looks for the method in class `B` before it looks for the mathod in the parent class of `A`.

Also important is `LayerizedNet.from_model`. It takes an existing model `model`, a list `layer_namess` of the names of the submodules that correspond to the layers, and two layer classes: the first one, `layer_base_type`, is the parent class for all layers except the last one, and the second one, `last_layer_type`, is the class of the last layer. The reason for this is, that it makes it very convenient to layerize models that use a certain nonlinear activation function in all but the last layer.

The method definition looks like this:
```Python
@classmethod
def from_model(cls, model, layer_names, layer_base_type, last_layer_type, *args, **kwargs):
    """
    model: nn.Module
    layer_names: list[string] contains the property names of the mappings that define the network
        eg: if model.forward does this:
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 16 * 5 * 5)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
        layer_names should look like this:
        ['conv1', 'pool', 'conv2', 'pool', 'fc1', 'fc2', 'fc3']
    layer_base_type: looks for layer classes that subclass this type to convert the model
    last_layer_type: type of the last layer
    *args, **kwargs: will be passed to the constructor of `cls`, i.e. LayerizedNet or a subclass

    Nonlinearities don't have to be specified if they are implemented in layer_base_type.
    Alternatively, they can be treated as extra layers.
    Reshaping should be implemented by the layer's .forward()
    """
```

## Create layers with extra logic
Let's load the `cifar10_utils` net so that we can create a layer scheme and a network class that adds some custom code that can make use of the layers.


In [2]:
%matplotlib inline
from cifar10_utils import *
net = Net()
net.load("cifar10net")
accuracy = net.accuracy(testloader)
print("Accuracy:", accuracy)

Files already downloaded and verified
Files already downloaded and verified
Accuracy: 0.5268


Now, we create a base layer and subclass from it, but also from the class that is imitated. For example, if the original network uses a `nn.Linear` module, `LayerizedNet.from_model` looks for a class that is both subclass of our custom `layer_base_type` and `nn.Linear`.

In [3]:
from layerized_net import *
from torch import nn
from torch.nn import functional as F
import numpy as np


class AMLayer(Layer):
    def custom_code(self, X):
        # add your additional code here, or overwride forward
        pass
   
class Relu():
    def forward(self, x):
        return F.relu(super().forward(x))
    
class Linear(AMLayer, nn.Linear):
    """
    For linear layers without Relu activation, usually the last
    """
    def forward(self, x):
        D = np.prod(x.shape[1:])
        x = x.view(-1, D)
        return super().forward(x)

class LinearRelu(Relu, Linear):
    pass

class ConvRelu(AMLayer, Relu, nn.Conv2d):
    pass

class MaxPool(AMLayer, Relu, nn.MaxPool2d):
    pass

class AMNet(LayerizedNet):
    def activation_maximization(self, layer, pattern):
        pass
    
    
layerized_net = AMNet.from_model(
    model=net,
    layer_names=['conv1', 'pool', 'conv2', 'pool', 'fc1', 'fc2', 'fc3'],
    layer_base_type=Relu, last_layer_type=Linear)

accuracy = layerized_net.accuracy(testloader)
print("Accuracy:", accuracy)

Accuracy: 0.5268


That's it, it worked!